# Introduction to spaCy

Copyright, Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

Whereas NLTK contains separate modules that need to be called individually, we now look at another toolkit that hides many steps in one processing module.

[spaCy](https://spacy.io/) provides a rather complete NLP pipeline (the output of one module feeds to the input of the next): it takes a raw document and performs tokenization, POS-tagging, stop word recognition, morphological analysis, lemmatization, sentence splitting, dependency parsing and Named Entity Recognition (NER). It also supports similarity prediction, but that is outside of the scope of this notebook. The advantage of spaCy is that it is really fast, and it has a good accuracy. In addition, it currently supports multiple languages, among which: English, German, Spanish, Portuguese, French, Italian and Dutch. 

So whereas in NLTK you built your own pipeline, spaCy gives you no choice and does it all for you.

In this notebook, we will show you the basic usage. If you want to learn more, please visit spaCy's website; it has extensive documentation and provides excellent user guides. 

**at the end of this notebook, you will be able to extract the output from spaCy for the following NLP tasks**:
* **Sentence splitting**: attribute **sents** of a `Doc` (of type *spacy.tokens.doc.Doc*)
* **Tokenization**: `Doc` contains a sequence of `Token` objects (of type *spacy.tokens.token.Token*)
* **Part-of-speech (POS) tagging**: attributes **pos_** and **tag_** of `Token`
* **Stop words recognition** attribute **is_stop** of `Token`
* **Stemming and lemmatization**: attribute **lemma_** of `Token`
* **Constituency/dependency parsing:** attributes **dep_** and **head**
* **Named Entity Recognition (NER):** attribute **ents** (of type *spacy.tokens.span.Span*) of `Doc` (of type *spacy.tokens.doc.Doc*). 

In addition, you will be able to use spaCy to visualize the output for each NLP task.


## Installing and loading spaCy

To install spaCy, check out the instructions [here](https://spacy.io/usage). On this page, it is explained exactly how to install spaCy for your operating system, package manager and desired languages. Simply run the suggested commands in your terminal ([Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) or cmd). Alternatively, you can probably also just run the following cells in this notebook:

**Tip**: comment out the next two commands after using them. You can comment out commands by putting a "#" in front.

In [ ]:
%%bash
conda install -c conda-forge spacy

Solving environment: ...working... done

## Package Plan ##

  environment location: /anaconda3

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.11               |           py36_0         3.0 MB  conda-forge
    conda-package-handling-1.4.1|           py36_0         261 KB  conda-forge
    libarchive-3.3.3           |       h786848e_5         1.5 MB
    lz4-2.1.9                  |   py36h9608888_0         115 KB  conda-forge
    lz4-c-1.8.3                |    h6de7cb9_1001         139 KB  conda-forge
    python-libarchive-c-2.8    |        py36_1004          21 KB  conda-forge
    zstd-1.3.7                 |    h06149cd_1001         589 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following NEW packages will be INSTALLED:

  conda-

In [ ]:
%%bash
python -m spacy download en

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=24fa5bca4b69a21cf3f5cca069a7b2432f57c9754a3035fff26d5ace39ca81f6
  Stored in directory: /private/var/folders/k1/bs53rbv901qc_hqgfx1cqm5c0000gn/T/pip-ephem-wheel-cache-ucxovngv/wheels/ee/4d/f7/563214122be1540b5f9197b52cb3ddb9c4a8070808b22d5a84
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-packages/en_core_web_sm
-->
/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


Now, let's first load spaCy. We import the spaCy module and load the English tokenizer, tagger, parser, NER, and word vectors.

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load('en') # other languages: de, es, pt, fr, it, nl

You might get a variation of the following error:
```
OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
```

This means that there is a problem with linking the language model of spaCy. You can try to load it in the following way:

In [ ]:
nlp = spacy.load('en_core_web_sm')

Even if this works, you might still want to invest some time to make sure that the linking was succesful, i.e., that you can load spaCy with spacy.load('en'). Here is some more information on how to fix this.

**Troubleshooting (optional)**

*Cause*: Anaconda prompt does not have enough priviliges to execute the linking part of `python -m spacy download en`. The same is true for any other `python -m spacy [...]`

*Solution:* 

1. Create the link manually

The prompt should display something along the lines of:

```
<Data Downloaded>
You do not have sufficient privilege to perform this operation.

    Linking successful
    <Anaconda dir>\lib\site-packages\en_core_web_sm -->
    <Anaconda dir>\lib\site-packages\spacy\data\en

    You can now load the model via spacy.load('en')

Use the following command:

mklink /D <Anaconda>\lib\site-packages\spacy\data\en <Anaconda>\lib\site-packages\en_core_web_sm

 Note that the target is pointing to the link, not the other way around. The syntax of the arguments is 'target' followed by 'link'
 
```

2. Give Anaconda Permissions to create link. Using "runas ... python -m spacy ..." may not suffice

3. More details: https://github.com/explosion/spaCy/issues/1283

**If none of this works (can happen on Windows)** You might want to install the model manually from SpaCy's GitHub through pip:

```
    pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz`.
```

## Using spaCy

If you succesfully loaded the English model (or another language), you now created the spaCy object 'nlp'. So what is an object? You can see it as an instance of a class of things. Whenever you create a variable and assign it a value of some type, this variable is a 'typed' instance. It has all the properties of this type and you can do the things with the variable that were defined for that type.

So now 'nlp' is an instance of a spaCy object loaded with the models for the English language. You can use 'nlp' now to process text through a defined pipeline of modules and store the result in another variable for accessing it. The results is another spaCy object of the type 'Doc'. 

'Doc' objects give you access to all the different analyses of the pipeline through different functions. In a Doc object you can access tokens, their lemmas, their PoS, sentences, chunks, named entities, etc.

Parsing a text with spaCy after loading a language model is as thus very easy. It works as follows, by passing your text as a string into the 'nlp' object.

In [ ]:
doc = nlp("I have an awesome cat. It's sitting on the mat that I bought yesterday.")

`doc` is now a Python object of the class `Doc`. It is a container for accessing linguistic annotations and a sequence of `Token` objects.

#### Doc, Token and Span objects

At this point, there are three important types of objects to remember:

* A `Doc` is a sequence of `Token` objects.
* A `Token` object represents an individual token — i.e. a word, punctuation symbol, whitespace, etc. It has attributes representing linguistic annotations. 
* A `Span` object is a slice from a `Doc` object and a sequence of `Token` objects.

Since `Doc` is a sequence of `Token` objects, we can iterate over all of the tokens in the text as shown below, or select a single token from the sequence. We need a 'for-loop' to iterate over the elements in the doc object.

In [ ]:
# Iterate over the tokens using a for loop
for token in doc:
    print(token)
print()


As usual in a list, we can access each token individually using the index:

In [ ]:
# Select one single token by index
first_token = doc[0]
print("First token:", first_token)

Please note that even though these look like strings, they are not. Print just gives the print representation of the token. We can also ask for the 'type' in Python and it will tell you what kind of object it is:

In [ ]:
for token in doc:
    print(token, "\t", type(token))

These `Token` objects have many useful methods and *attributes*, which we can list by using `dir()`. We haven't really talked about attributes during this course, but while methods are operations or activities performed by that object, attributes are 'static' features of the objects. Methods are called using parantheses (as we have seen with `str.upper()`, for instance), while attributes are indicated without parentheses. We will see some examples below.

You can find more detailed information about the token methods and attributes in the [documentation](https://spacy.io/api/token).

In [ ]:
dir(first_token)

Let's inspect some of the attributes of the tokens. Can you figure out what they mean? Feel free to try out a few more.

In [ ]:
# Print attributes of tokens
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_)

Notice that some of the attributes end with an underscore. For example, tokens have both `lemma` and `lemma_` attributes. The `lemma` attribute represents the id of the lemma (integer), while the `lemma_` attribute represents the unicode string representation of the lemma. In practice, you will mostly use the `lemma_` attribute.

In [ ]:
for token in doc:
    print(token.lemma, token.lemma_)

You can also use spacy.explain to find out more about certain labels:

In [ ]:
# try out some more, such as NN, ADP, PRP, VBD, VBP, VBZ, WDT, aux, nsubj, pobj, dobj, npadvmod
spacy.explain("PRON")

## Sentence splitting & tokenization
spaCy performs sentence splitting for you. The information is stored in the attribute **sents** of `Doc` (of type *spacy.tokens.doc.Doc*).
Each `Doc` contains a sequence of `Token` objects, i.e., this is where the output from the tokenizer is found. The token itself can be accessed using the attribute **text**.

In [ ]:
doc = nlp("I have an awesome cat. It's sitting on the mat that I bought yesterday.")

In [ ]:
sentences=doc.sents
for sentence in sentences:
    print()
    print(sentence)
    for token in sentence:
        print(token.text)

## Lemmatization
The output from the lemmatizer is stored in the attribute **lemma_** of each `Token' object.

In [ ]:
doc = nlp("I have awesome cats")

In [ ]:
cat_token = doc[3]
print(cat_token.text, cat_token.lemma_)

## Part of speech tagging

The output from the part of speech tagger is stored:
* in the attribute **pos_** of each `Token` object: The simple part-of-speech tag
* in the attribute **tag_** of each `Token object: The detailed part-of-speech tag ([Penn Treebank POS tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html))

In [ ]:
doc = nlp("I have awesome cats")

In [ ]:
cat_token = doc[3]
print(cat_token.text, cat_token.pos_, cat_token.tag_)
spacy.explain(cat_token.tag_)

## Dependency parsing
The output of the dependency parser can only be accessed by combining the information from multiple attributes. Let's look at an example:

In [ ]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")

In [ ]:
displacy.render(doc, jupyter=True, style='dep')

We observe that each token has a dependency relation with at least one other token. For example:
* **cars** has an **amod** relation with **autonomous**
* **shift** has an **nsubj** relation with **cars**

If you want to know what these relations mean, you can use **spacy.explain**

In [ ]:
spacy.explain('amod')

spaCy makes use of the terms **child** and **head** in their dependency parsing output.
* a relation is always between a **child** and a **head**, e.g., *autonomous* is the child of *cars*
* a head of a phrase can be the child of another token, e.g., *cars* is the child of *shift*

The following attributes are needed to access this information:
* **dep_** provides the syntactic relation, e.g., *nsubj*
* **head** provides the **head** of a `Token`, e.g., in the case of *autonomous* the head would be *cars*

In [ ]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")

In [ ]:
autonomous_token = doc[0]
print(autonomous_token.dep_, autonomous_token.head)

In [ ]:
cars_token = doc[1]
print(cars_token.text, cars_token.head)

### save tree structure to SVG image

In [ ]:
tree_structure = displacy.render(doc, jupyter=False, style='dep')

output_path = 'spacy_tree_structure.svg'
with open(output_path, 'w') as outfile:
    outfile.write(tree_structure)

## Named Entity Recognition
The output from the Named Entity Recognizer is stored in the attribute **ents** of `Doc`.
The attribute **label_** and an **ent** (of type *spacy.tokens.span.Span*) contains the named entity type.

In [ ]:
text = """But Google is starting from behind. The company made a late push into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa software, which runs on its Echo and Dot devices, have clear leads in consumer adoption."""
doc = nlp(text)

In [ ]:
displacy.render(doc, jupyter=True, style='ent')

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

# End of this notebook